In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install dominate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install visdom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 4.2 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408217 sha256=a08ed92657109eccef81f98a8c73fc53f819dbaf6ba2da9d6f22f30a5566eeba
  Stored in directory: /root/.cache/pip/wheels/58/9e/14/30f7cc4dafdd4d602fb00ca33c6edd1424fc0f5df10a02e060
Successfully built visdom


In [ ]:
!git clone https://github.com/faisalml/DeepLSR

Cloning into 'DeepLSR'...
remote: Enumerating objects: 186, done.
remote: Total 186 (delta 0), reused 0 (delta 0), pack-reused 186
Receiving objects: 100% (186/186), 5.37 MiB | 5.14 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [ ]:
%cd DeepLSR

/content/DeepLSR


In [ ]:
!find . -name "*.pyc" -delete

In [ ]:
!cp /content/drive/MyDrive/projet_imagerie/data/personal_data.zip /content/personal_data.zip
!unzip /content/personal_data.zip 

Archive:  /content/personal_data.zip
replace personal_data/base/09-41-06_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: personal_data/base/09-41-06_1.jpg  
  inflating: personal_data/base/09-41-06_2.jpg  
  inflating: personal_data/base/09-42-05_1.jpg  
  inflating: personal_data/base/09-42-05_2.jpg  
  inflating: personal_data/base/09-43-07_1.jpg  
  inflating: personal_data/base/09-43-07_2.jpg  
  inflating: personal_data/base/09-44-06_1.jpg  
  inflating: personal_data/base/09-44-06_2.jpg  
  inflating: personal_data/base/09-45-04_1.jpg  
  inflating: personal_data/base/09-45-04_2.jpg  
  inflating: personal_data/base/09-46-10_1.jpg  
  inflating: personal_data/base/09-46-10_2.jpg  
  inflating: personal_data/base/09-49-17_1.jpg  
  inflating: personal_data/base/09-49-17_2.jpg  
  inflating: personal_data/base/09-49-37_1.jpg  
  inflating: personal_data/base/09-49-37_2.jpg  
  inflating: personal_data/base/09-50-11_1.jpg  
  inflating: personal_data/base/09-50-11_2.jpg 

In [ ]:
!cp /content/drive/MyDrive/projet_imagerie/DLSR/DeepLSR_Model_Release.zip /content/DeepLSR_Model_Release.zip
!unzip /content/DeepLSR_Model_Release.zip

Archive:  /content/DeepLSR_Model_Release.zip
mapname:  conversion of  failed
replace LSR2LED/latest_net_G.pth? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!cp  /content/DeepLSR/LSR2LED/* /content/DeepLSR/checkpoints/DeepLSR/ 

In [ ]:
!python test.py --dataroot /content/DeepLSR/personal_data/speckle --name DeepLSR --gpu_ids 0 --display_id 0

----------------- Options ---------------
             aspect_ratio: 1.0                           
                batchSize: 1                             
          checkpoints_dir: ./checkpoints                 
                 dataroot: /content/DeepLSR/personal_data/speckle	[default: None]
             dataset_mode: single                        
               display_id: 0                             	[default: 1]
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                 fineSize: 256                           
                  gpu_ids: 0                             
                 how_many: 50                            
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                   

In [ ]:
!cp /content/DeepLSR/results/DeepLSR/test_latest/images/* /content/drive/MyDrive/projet_imagerie/DLSR/Images/breast_cancer 

**REPLACER LE FICHIER visualizer.py par le contenu-ci dessous**

```py
import numpy as np
import os
import ntpath
import time
from . import util
from . import html
from skimage.transform import resize


# save image to the disk
def save_images(webpage, visuals, image_path, aspect_ratio=1.0, width=256):
    image_dir = webpage.get_image_dir()
    short_path = ntpath.basename(image_path[0])
    name = os.path.splitext(short_path)[0]

    webpage.add_header(name)
    ims, txts, links = [], [], []

    for label, im_data in visuals.items():
        im = util.tensor2im(im_data)
        image_name = '%s_%s.png' % (name, label)
        save_path = os.path.join(image_dir, image_name)
        h, w, _ = im.shape
        if aspect_ratio > 1.0:
            im = resize(im, (h, int(w * aspect_ratio)), interp='bicubic')
        if aspect_ratio < 1.0:
            im = resize(im, (int(h / aspect_ratio), w), interp='bicubic')
        util.save_image(im, save_path)

        ims.append(image_name)
        txts.append(label)
        links.append(image_name)
    webpage.add_images(ims, txts, links, width=width)


class Visualizer():
    def __init__(self, opt):
        self.display_id = opt.display_id
        self.use_html = opt.isTrain and not opt.no_html
        self.win_size = opt.display_winsize
        self.name = opt.name
        self.opt = opt
        self.saved = False
        if self.display_id > 0:
            import visdom
            self.ncols = opt.display_ncols
            self.vis = visdom.Visdom(server=opt.display_server, port=opt.display_port, env=opt.display_env, raise_exceptions=True)

        if self.use_html:
            self.web_dir = os.path.join(opt.checkpoints_dir, opt.name, 'web')
            self.img_dir = os.path.join(self.web_dir, 'images')
            print('create web directory %s...' % self.web_dir)
            util.mkdirs([self.web_dir, self.img_dir])
        self.log_name = os.path.join(opt.checkpoints_dir, opt.name, 'loss_log.txt')
        with open(self.log_name, "a") as log_file:
            now = time.strftime("%c")
            log_file.write('================ Training Loss (%s) ================\n' % now)

    def reset(self):
        self.saved = False

    def throw_visdom_connection_error(self):
        print('\n\nCould not connect to Visdom server (https://github.com/facebookresearch/visdom) for displaying training progress.\nYou can suppress connection to Visdom using the option --display_id -1. To install visdom, run \n$ pip install visdom\n, and start the server by \n$ python -m visdom.server.\n\n')
        exit(1)

    # |visuals|: dictionary of images to display or save
    def display_current_results(self, visuals, epoch, save_result):
        if self.display_id > 0:  # show images in the browser
            ncols = self.ncols
            if ncols > 0:
                ncols = min(ncols, len(visuals))
                h, w = next(iter(visuals.values())).shape[:2]
                table_css = """<style>
                        table {border-collapse: separate; border-spacing:4px; white-space:nowrap; text-align:center}
                        table td {width: %dpx; height: %dpx; padding: 4px; outline: 4px solid black}
                        </style>""" % (w, h)
                title = self.name
                label_html = ''
                label_html_row = ''
                images = []
                idx = 0
                for label, image in visuals.items():
                    image_numpy = util.tensor2im(image)
                    label_html_row += '<td>%s</td>' % label
                    images.append(image_numpy.transpose([2, 0, 1]))
                    idx += 1
                    if idx % ncols == 0:
                        label_html += '<tr>%s</tr>' % label_html_row
                        label_html_row = ''
                white_image = np.ones_like(image_numpy.transpose([2, 0, 1])) * 255
                while idx % ncols != 0:
                    images.append(white_image)
                    label_html_row += '<td></td>'
                    idx += 1
                if label_html_row != '':
                    label_html += '<tr>%s</tr>' % label_html_row
                # pane col = image row
                try:
                    self.vis.images(images, nrow=ncols, win=self.display_id + 1,
                                    padding=2, opts=dict(title=title + ' images'))
                    label_html = '<table>%s</table>' % label_html
                    self.vis.text(table_css + label_html, win=self.display_id + 2,
                                  opts=dict(title=title + ' labels'))
                except ConnectionError:
                    self.throw_visdom_connection_error()

            else:
                idx = 1
                for label, image in visuals.items():
                    image_numpy = util.tensor2im(image)
                    self.vis.image(image_numpy.transpose([2, 0, 1]), opts=dict(title=label),
                                   win=self.display_id + idx)
                    idx += 1

        if self.use_html and (save_result or not self.saved):  # save images to a html file
            self.saved = True
            for label, image in visuals.items():
                image_numpy = util.tensor2im(image)
                img_path = os.path.join(self.img_dir, 'epoch%.3d_%s.png' % (epoch, label))
                util.save_image(image_numpy, img_path)
            # update website
            webpage = html.HTML(self.web_dir, 'Experiment name = %s' % self.name, reflesh=1)
            for n in range(epoch, 0, -1):
                webpage.add_header('epoch [%d]' % n)
                ims, txts, links = [], [], []

                for label, image_numpy in visuals.items():
                    image_numpy = util.tensor2im(image)
                    img_path = 'epoch%.3d_%s.png' % (n, label)
                    ims.append(img_path)
                    txts.append(label)
                    links.append(img_path)
                webpage.add_images(ims, txts, links, width=self.win_size)
            webpage.save()

    # losses: dictionary of error labels and values
    def plot_current_losses(self, epoch, counter_ratio, opt, losses):
        if not hasattr(self, 'plot_data'):
            self.plot_data = {'X': [], 'Y': [], 'legend': list(losses.keys())}
        self.plot_data['X'].append(epoch + counter_ratio)
        self.plot_data['Y'].append([losses[k] for k in self.plot_data['legend']])
        try:
            self.vis.line(
                X=np.stack([np.array(self.plot_data['X'])] * len(self.plot_data['legend']), 1),
                Y=np.array(self.plot_data['Y']),
                opts={
                    'title': self.name + ' loss over time',
                    'legend': self.plot_data['legend'],
                    'xlabel': 'epoch',
                    'ylabel': 'loss'},
                win=self.display_id)
        except ConnectionError:
            self.throw_visdom_connection_error()

    # losses: same format as |losses| of plot_current_losses
    def print_current_losses(self, epoch, i, losses, t, t_data):
        message = '(epoch: %d, iters: %d, time: %.3f, data: %.3f) ' % (epoch, i, t, t_data)
        for k, v in losses.items():
            message += '%s: %.3f ' % (k, v)

        print(message)
        with open(self.log_name, "a") as log_file:
            log_file.write('%s\n' % message)
```